# 3. Guardrail

이 노트북에서는 LangChain agent의 guardrail 기능을 확인해보겠습니다.


In [ ]:
import sys
import os
from dotenv import load_dotenv

# 1. 현재 작업 디렉토리 설정
current_dir = os.getcwd()
project_root = os.path.abspath(os.path.join(current_dir, "llmops_class"))

# 2. sys.path에 추가 (이제 Python이 'app' 폴더를 찾을 수 있습니다)
if project_root not in sys.path:
    sys.path.append(project_root)
    print(f"✅ Project Root added to sys.path: {project_root}")

# 3. .env 파일 로드
dotenv_path = os.path.join("llmops_class", ".env")

if os.path.exists(dotenv_path):
    print(f"Loading .env from: {dotenv_path}")
    load_dotenv(dotenv_path)
    
    # LangSmith Project Setting (Server Specific)
    # .env에는 API Key만 있고, 프로젝트명은 여기서 분리합니다.
    os.environ["LANGSMITH_TRACING"] = "true"

In [34]:
from langchain.agents import create_agent
from langchain.agents.middleware import PIIMiddleware

agent = create_agent(
    model="gpt-4o",
    tools=[],
    middleware=[
        # 사용자 입력에서 이메일을 모델로 보내기 전에 삭제(redact) 처리
        PIIMiddleware(
            "email",
            strategy="redact",
            apply_to_input=True,
        ),
        # 사용자 입력에서 신용카드 번호를 마스킹(mask) 처리 (예: ****-****-****-1234)
        PIIMiddleware(
            "credit_card",
            strategy="mask",
            apply_to_input=True,
        ),
        # API 키가 감지되면 차단(block)하고 에러 발생
        PIIMiddleware(
            "api_key",
            detector=r"sk-[a-zA-Z0-9]{32}",
            strategy="block",
            apply_to_input=True,
        ),
    ],
)


In [35]:
def run_test(title, user_input):
    print(f"--- [테스트: {title}] ---")
    print(f"입력 메시지: {user_input}")
    try:
        # 에이전트 실행
        result = agent.invoke({
            "messages": [{"role": "user", "content": user_input}]
        })

        # 결과 출력 (모델은 가드레일이 처리한 내용을 바탕으로 응답합니다)
        print(f"\n🤖 모델 응답: {result['messages'][-1].content}")
        print(">>> 정상 처리됨 (Redact/Mask 전략 성공)")

    except Exception as e:
        # Block 전략이 실행되면 에러가 발생합니다.
        print(f"\n⛔ 가드레일 차단 발생 (Block 전략 성공)!")
        print(f"에러 메시지: {e}")
    print("-" * 30 + "\n")


# --- 3. 실제 테스트 실행 ---

# 시나리오 1: 이메일과 신용카드 정보가 포함된 경우 (Redact & Mask 테스트)
# 예상: 이메일은 삭제되고, 카드번호는 마스킹된 채로 모델이 응답해야 함.
input_pii = "제 이메일은 user@example.com 이고, 결제할 카드 번호는 4111-2222-3333-4444 입니다. 확인해주세요."
run_test("PII 정보 포함 (이메일/카드)", input_pii)


# 시나리오 2: 민감한 API 키가 포함된 경우 (Block 테스트)
# 예상: 미들웨어가 이를 감지하고 에러를 발생시켜 실행을 중단해야 함.
# (테스트용 가짜 키입니다)
input_apikey = "혹시 이 API 키가 유효한가요? sk-aBcDeFgHiJkLmNoPqRsTuVwXyZ0123456789 입니다."
run_test("API 키 유출 시도", input_apikey)


# 시나리오 3: 민감한 정보가 없는 일반적인 대화
# 예상: 아무런 변경 없이 정상적으로 대화가 진행되어야 함.
input_normal = "안녕하세요, 오늘 날씨가 참 좋네요."
run_test("일반적인 대화", input_normal)

--- [테스트: PII 정보 포함 (이메일/카드)] ---
입력 메시지: 제 이메일은 user@example.com 이고, 결제할 카드 번호는 4111-2222-3333-4444 입니다. 확인해주세요.

🤖 모델 응답: 죄송하지만, 이메일 주소나 카드 번호와 같은 민감한 개인 정보를 수신하거나 저장할 수 없습니다. 이러한 정보를 안전하게 취급하는 것이 중요하므로, 필요한 경우 신뢰할 수 있는 보안된 경로를 통해 정보를 공유하시기 바랍니다. 도움이 필요하시면 일반적인 질문으로 도와드리겠습니다.
>>> 정상 처리됨 (Redact/Mask 전략 성공)
------------------------------

--- [테스트: API 키 유출 시도] ---
입력 메시지: 혹시 이 API 키가 유효한가요? sk-aBcDeFgHiJkLmNoPqRsTuVwXyZ0123456789 입니다.

⛔ 가드레일 차단 발생 (Block 전략 성공)!
에러 메시지: Detected 1 instance(s) of api_key in text content
------------------------------

--- [테스트: 일반적인 대화] ---
입력 메시지: 안녕하세요, 오늘 날씨가 참 좋네요.

🤖 모델 응답: 안녕하세요! 오늘 날씨가 좋다니 기분도 좋아지네요. 밖에 나가서 산책하거나 활동하기에 딱 좋은 날씨인가 봐요. 특별히 계획하신 일이 있으신가요?
>>> 정상 처리됨 (Redact/Mask 전략 성공)
------------------------------

